In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch import optim
import scipy.sparse as sp
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
import time
from tqdm.auto import tqdm

# Set seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

print("✅ All imports successful | Seeds set to 42")

# ============================================================================
# CONFIG (tune here)
# ============================================================================

config = {
    # Model
    "embedding_dim": 128,  # try 64/128/256
    "n_layers": 3,         # try 2/3/4

    # Training
    "epochs": 200,         # try 100-300 on T4
    "batch_size": 4096,    # try 2048/4096/8192
    "lr": 1e-3,            # try 5e-4, 1e-3, 2e-3
    "reg": 1e-5,           # try 1e-4, 5e-5, 1e-5, 5e-6
    "n_negs": 5,           # try 1/5/10

    # Validation / early stopping
    "val_ratio": 0.10,     # most recent 10% per user from train_df
    "min_user_train_for_val": 6,  # only split users with >= this many train interactions
    "eval_every": 5,       # validate every N epochs
    "patience": 30,        # try 20-50
}

# ============================================================================
# DATA LOADING & PREPROCESSING
# ============================================================================

print("\n" + "="*70)
print("LOADING DATA")
print("="*70)

ratings = pd.read_csv('ratings.dat', sep='::', engine='python',
                      names=['userId', 'movieId', 'rating', 'timestamp'])
ratings['rating'] = 1  # Implicit feedback

user_encoder = LabelEncoder()
item_encoder = LabelEncoder()
ratings['user'] = user_encoder.fit_transform(ratings['userId'])
ratings['item'] = item_encoder.fit_transform(ratings['movieId'])

n_users = ratings['user'].nunique()
n_items = ratings['item'].nunique()
print(f"📊 Users: {n_users} | Items: {n_items} | Interactions: {len(ratings):,}")

# ============================================================================
# 80:20 TRAIN-TEST SPLIT (Per-User Interaction Level)
# ============================================================================

print("\n" + "="*70)
print("80:20 TRAIN-TEST SPLIT (PER USER)")
print("="*70)

train_list = []
test_list = []

users_with_5_or_less = 0
users_in_test = 0

for user_id in range(n_users):
    user_data = ratings[ratings['user'] == user_id].sort_values('timestamp')
    n_interactions = len(user_data)

    if n_interactions > 5:
        # Apply 80:20 split for users with >5 interactions
        split_idx = int(0.8 * n_interactions)
        train_list.append(user_data.iloc[:split_idx])
        test_list.append(user_data.iloc[split_idx:])
        users_in_test += 1
    else:
        # Keep all in training for users with ≤5 interactions
        train_list.append(user_data)
        users_with_5_or_less += 1

train_df = pd.concat(train_list, ignore_index=True)
test_df = pd.concat(test_list, ignore_index=True) if test_list else pd.DataFrame()

print(f"📈 Train: {len(train_df):,} | Test: {len(test_df):,}")
print(f"👥 Users with ≤5 interactions (training only): {users_with_5_or_less}")
print(f"👥 Users in test set: {users_in_test}")

# ============================================================================
# ITEM-AWARE SPLIT FIX (Strategy B: ensure every item appears in train)
# ============================================================================

print("\n" + "="*70)
print("ITEM-AWARE SPLIT FIX (ENSURE ALL ITEMS IN TRAIN)")
print("="*70)

train_items_set = set(train_df['item'].unique())
cold_items = sorted(set(range(n_items)) - train_items_set)
print(f"🧊 Items missing in train (cold items): {len(cold_items)}")

moved_rows = 0
if len(cold_items) > 0 and len(test_df) > 0:
    # Move one (earliest) test interaction per cold item into train
    move_idx = []
    for it in cold_items:
        candidates = test_df[test_df['item'] == it]
        if len(candidates) == 0:
            continue
        idx = candidates['timestamp'].idxmin()
        move_idx.append(idx)

    move_idx = sorted(set(move_idx))
    moved_rows = len(move_idx)
    if moved_rows > 0:
        train_df = pd.concat([train_df, test_df.loc[move_idx]], ignore_index=True)
        test_df = test_df.drop(index=move_idx).reset_index(drop=True)

missing_after = set(range(n_items)) - set(train_df['item'].unique())
if len(missing_after) == 0:
    print("✓ All items appear in training after item-aware fix")
else:
    print(f"⚠️  Still missing {len(missing_after)} items in train (cold-start remains)")

print(f"🔁 Moved {moved_rows} interactions from test → train")
print(f"📈 Updated Train: {len(train_df):,} | Updated Test: {len(test_df):,}")

# ============================================================================
# VALIDATION SPLIT (from train_df only) - do NOT tune on test
# ============================================================================

print("\n" + "="*70)
print("VALIDATION SPLIT (FROM TRAIN ONLY)")
print("="*70)

val_ratio = float(config["val_ratio"])
min_user_train_for_val = int(config["min_user_train_for_val"])

train2_list = []
val_list = []

for user_id in range(n_users):
    udata = train_df[train_df['user'] == user_id].sort_values('timestamp')
    n = len(udata)

    if n >= min_user_train_for_val:
        val_n = int(np.ceil(val_ratio * n))
        val_n = min(val_n, n - 1)  # keep at least 1 training interaction
        if val_n > 0:
            train2_list.append(udata.iloc[:-val_n])
            val_list.append(udata.iloc[-val_n:])
        else:
            train2_list.append(udata)
    else:
        train2_list.append(udata)

train_df2 = pd.concat(train2_list, ignore_index=True)
val_df = pd.concat(val_list, ignore_index=True) if len(val_list) > 0 else pd.DataFrame()

print(f"📈 Train2 (used for training): {len(train_df2):,}")
print(f"🧪 Val (used for early stopping): {len(val_df):,}")

# Ensure we didn't reintroduce cold items by moving the only occurrence to val
missing_items_after_val = sorted(set(range(n_items)) - set(train_df2['item'].unique()))
if len(missing_items_after_val) > 0 and len(val_df) > 0:
    print(f"⚠️  {len(missing_items_after_val)} items became cold after val holdout; moving 1 val row/item back to train")
    move_back_idx = []
    for it in missing_items_after_val:
        candidates = val_df[val_df['item'] == it]
        if len(candidates) == 0:
            continue
        idx = candidates['timestamp'].idxmin()
        move_back_idx.append(idx)

    move_back_idx = sorted(set(move_back_idx))
    if len(move_back_idx) > 0:
        train_df2 = pd.concat([train_df2, val_df.loc[move_back_idx]], ignore_index=True)
        val_df = val_df.drop(index=move_back_idx).reset_index(drop=True)

missing_items_after_val_fix = set(range(n_items)) - set(train_df2['item'].unique())
if len(missing_items_after_val_fix) == 0:
    print("✓ All items appear in Train2 after val adjustment")
else:
    print(f"⚠️  Still missing {len(missing_items_after_val_fix)} items in Train2 (isolated nodes may remain)")

# Save splits
train_df2.to_csv('train_split.csv', index=False)
val_df.to_csv('val_split.csv', index=False)
test_df.to_csv('test_split.csv', index=False)
print("\n💾 Saved: train_split.csv, val_split.csv, test_split.csv")

# ============================================================================
# POST-SPLIT SANITY CHECKS
# ============================================================================

print("\n" + "="*70)
print("POST-SPLIT SANITY CHECKS")
print("="*70)

train_users = train_df2['user'].unique()
train_items = train_df2['item'].unique()
print(f"✓ Training users (Train2): {len(train_users)}/{n_users}")
print(f"✓ Training items (Train2): {len(train_items)}/{n_items}")

val_users = val_df['user'].unique() if len(val_df) > 0 else []
val_users_without_train = set(val_users) - set(train_users)
if len(val_users_without_train) > 0:
    print(f"⚠️  WARNING: {len(val_users_without_train)} val users have no training data!")
else:
    print("✓ All val users have training data")

test_users = test_df['user'].unique() if len(test_df) > 0 else []
test_users_without_train = set(test_users) - set(train_users)
if len(test_users_without_train) > 0:
    print(f"⚠️  WARNING: {len(test_users_without_train)} test users have no training data!")
else:
    print("✓ All test users have training data")

# Verify no leakage between Train2 and Test
train_interactions = set(zip(train_df2['user'].values, train_df2['item'].values))
test_interactions = set(zip(test_df['user'].values, test_df['item'].values)) if len(test_df) > 0 else set()
leakage = train_interactions & test_interactions
if len(leakage) > 0:
    print(f"⚠️  WARNING: {len(leakage)} interactions leaked from test to train!")
else:
    print("✓ No data leakage detected (Train2-Test disjoint)")

# ============================================================================
# ADJACENCY MATRIX CONSTRUCTION (built only from Train2)
# ============================================================================

def build_adj_matrix(df, n_users, n_items):
    """Build normalized adjacency: D^(-1/2) * A * D^(-1/2)"""
    user_item_matrix = sp.coo_matrix(
        (df['rating'].values, (df['user'].values, df['item'].values)),
        shape=(n_users, n_items)
    )

    R = sp.vstack([
        sp.hstack([sp.csr_matrix((n_users, n_users)), user_item_matrix]),
        sp.hstack([user_item_matrix.T, sp.csr_matrix((n_items, n_items))])
    ])

    rowsum = np.array(R.sum(1)).flatten()
    # Avoid divide-by-zero warnings for isolated nodes (degree = 0)
    d_inv_sqrt = np.zeros_like(rowsum)
    nonzero = rowsum > 0
    d_inv_sqrt[nonzero] = np.power(rowsum[nonzero], -0.5)
    d_mat_inv_sqrt = sp.diags(d_inv_sqrt)
    norm_adj = d_mat_inv_sqrt.dot(R).dot(d_mat_inv_sqrt).tocoo()

    return norm_adj


def convert_sp_mat_to_sp_tensor(X):
    """Convert scipy sparse to PyTorch sparse tensor"""
    X = X.tocoo()
    indices = torch.LongTensor([X.row, X.col])
    values = torch.FloatTensor(X.data)
    tensor = torch.sparse.FloatTensor(indices, values, torch.Size(X.shape))
    return tensor.coalesce()  # ✅ CRITICAL: Coalesce for correctness


print("\n" + "="*70)
print("BUILDING GRAPH ADJACENCY")
print("="*70)

train_adj_matrix = build_adj_matrix(train_df2, n_users, n_items)
print(f"🔗 Adjacency: {train_adj_matrix.shape} | Edges: {train_adj_matrix.nnz:,}")

# ============================================================================
# LIGHTGCN MODEL
# ============================================================================


class LightGCN(nn.Module):
    def __init__(self, n_users, n_items, embedding_dim=64, n_layers=3):
        super().__init__()
        self.n_users = n_users
        self.n_items = n_items
        self.embedding_dim = embedding_dim
        self.n_layers = n_layers

        self.user_embedding = nn.Embedding(n_users, embedding_dim)
        self.item_embedding = nn.Embedding(n_items, embedding_dim)
        nn.init.xavier_uniform_(self.user_embedding.weight)
        nn.init.xavier_uniform_(self.item_embedding.weight)

    def forward(self, adj):
        """
        ✅ CORRECT: Graph propagation through multiple layers
        Computes layer-wise embeddings and returns their mean
        """
        all_embeddings = torch.cat([self.user_embedding.weight, self.item_embedding.weight], dim=0)
        embs = [all_embeddings]

        for _ in range(self.n_layers):
            all_embeddings = torch.sparse.mm(adj, all_embeddings)
            embs.append(all_embeddings)

        embs = torch.stack(embs, dim=1).mean(dim=1)
        return embs[:self.n_users], embs[self.n_users:]

    def get_ego_embeddings(self):
        """✅ Return layer-0 embeddings for regularization"""
        return self.user_embedding.weight, self.item_embedding.weight


print("\n" + "="*70)
print("MODEL INITIALIZATION")
print("="*70)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🖥️  Device: {device}")

train_adj_torch = convert_sp_mat_to_sp_tensor(train_adj_matrix).to(device)
model = LightGCN(
    n_users,
    n_items,
    embedding_dim=int(config["embedding_dim"]),
    n_layers=int(config["n_layers"]),
).to(device)

print(f"📐 Embeddings: {config['embedding_dim']}-dim | Layers: {config['n_layers']}")
print(f"🔢 Parameters: {sum(p.numel() for p in model.parameters()):,}")

# ============================================================================
# TRAINING PREPARATION (from Train2 only)
# ============================================================================

# Build user -> positive items mapping (Train2 only)
user_pos = defaultdict(set)
for u, i in zip(train_df2['user'].values, train_df2['item'].values):
    user_pos[int(u)].add(int(i))

user_item_pairs = np.array(list(zip(train_df2['user'].values, train_df2['item'].values)), dtype=np.int64)

print(f"\n📊 Training pairs (Train2): {len(user_item_pairs):,}")


def sample_neg_vectorized(users, n_items, user_pos, n_negs=1):
    """Fast negative sampling with rejection sampling (O(1) expected time).

    Returns:
      - (batch,) if n_negs==1
      - (batch, n_negs) if n_negs>1
    """
    users = np.asarray(users, dtype=np.int64)
    n_negs = int(n_negs)

    if n_negs <= 1:
        neg_items = []
        for u in users:
            pos_set = user_pos[int(u)]
            while True:
                neg = np.random.randint(0, n_items)
                if neg not in pos_set:
                    neg_items.append(neg)
                    break
        return np.asarray(neg_items, dtype=np.int64)

    neg_items = np.empty((len(users), n_negs), dtype=np.int64)
    for row, u in enumerate(users):
        pos_set = user_pos[int(u)]
        for k in range(n_negs):
            while True:
                neg = np.random.randint(0, n_items)
                if neg not in pos_set:
                    neg_items[row, k] = neg
                    break
    return neg_items


def bpr_loss_multi_neg(
    users,
    pos_items,
    neg_items,
    user_emb,
    item_emb,
    user_emb_ego,
    item_emb_ego,
    reg_weight=1e-4,
):
    """BPR loss with optional multi-negative sampling.

    Regularization is applied to layer-0 (ego) embeddings only.

    Shapes:
      users: (B,)
      pos_items: (B,)
      neg_items: (B,) or (B, N)
    """
    u_emb = user_emb[users]           # (B, D)
    pos_emb = item_emb[pos_items]     # (B, D)

    pos_scores = torch.sum(u_emb * pos_emb, dim=1)  # (B,)

    if neg_items.dim() == 1:
        neg_emb = item_emb[neg_items]  # (B, D)
        neg_scores = torch.sum(u_emb * neg_emb, dim=1)  # (B,)
        bpr = -torch.mean(torch.log(torch.sigmoid(pos_scores - neg_scores) + 1e-10))

        # Reg: ego embeddings only
        u_ego = user_emb_ego[users]
        pos_ego = item_emb_ego[pos_items]
        neg_ego = item_emb_ego[neg_items]

        reg_loss = reg_weight * (
            torch.sum(u_ego ** 2) + torch.sum(pos_ego ** 2) + torch.sum(neg_ego ** 2)
        ) / users.shape[0]

        return bpr + reg_loss

    # Multi-negatives: neg_items is (B, N)
    neg_emb = item_emb[neg_items]  # (B, N, D)
    u_emb_exp = u_emb.unsqueeze(1)  # (B, 1, D)
    neg_scores = torch.sum(u_emb_exp * neg_emb, dim=2)  # (B, N)

    pos_scores_exp = pos_scores.unsqueeze(1)  # (B, 1)
    bpr = -torch.mean(torch.log(torch.sigmoid(pos_scores_exp - neg_scores) + 1e-10))

    # Reg: ego embeddings only
    u_ego = user_emb_ego[users]
    pos_ego = item_emb_ego[pos_items]
    neg_ego = item_emb_ego[neg_items]  # (B, N, D)

    reg_loss = reg_weight * (
        torch.sum(u_ego ** 2) + torch.sum(pos_ego ** 2) + torch.sum(neg_ego ** 2)
    ) / users.shape[0]

    return bpr + reg_loss


def build_items_dict(df):
    """user -> set(items)"""
    d = defaultdict(set)
    if df is None or len(df) == 0:
        return d
    for u, i in zip(df['user'].values, df['item'].values):
        d[int(u)].add(int(i))
    return d


def evaluate_topk(user_emb, item_emb, test_items, train_items, top_k=20):
    """Leakage-free ranking metrics with train exclusion.

    test_items: dict[user] -> set(gt items)
    train_items: dict[user] -> set(items to mask)
    """
    hits = []
    precisions = []
    recalls = []
    ndcgs = []

    for user in test_items:
        if user >= len(user_emb):
            continue

        scores = np.dot(item_emb, user_emb[user])

        train_mask = list(train_items[user])
        if len(train_mask) > 0:
            scores[train_mask] = -np.inf

        top_items = np.argsort(scores)[::-1][:top_k]
        ground_truth = test_items[user]

        hits_in_topk = len(set(top_items) & ground_truth)
        hits.append(1 if hits_in_topk > 0 else 0)
        precisions.append(hits_in_topk / top_k)
        recalls.append(hits_in_topk / len(ground_truth) if len(ground_truth) > 0 else 0)

        dcg = sum([1.0 / np.log2(i + 2) for i, item in enumerate(top_items) if item in ground_truth])
        idcg = sum([1.0 / np.log2(i + 2) for i in range(min(len(ground_truth), top_k))])
        ndcgs.append(dcg / idcg if idcg > 0 else 0.0)

    return {
        'HR': float(np.mean(hits)) if len(hits) > 0 else 0.0,
        'Precision': float(np.mean(precisions)) if len(precisions) > 0 else 0.0,
        'Recall': float(np.mean(recalls)) if len(recalls) > 0 else 0.0,
        'NDCG': float(np.mean(ndcgs)) if len(ndcgs) > 0 else 0.0,
    }


val_items_dict = build_items_dict(val_df)

# ============================================================================
# TRAINING LOOP
# ============================================================================

print("\n" + "="*70)
print("TRAINING")
print("="*70)

epochs = int(config["epochs"])
batch_size = int(config["batch_size"])
lr = float(config["lr"])
reg = float(config["reg"])
n_negs = int(config["n_negs"])
optimizer = optim.Adam(model.parameters(), lr=lr)

eval_every = int(config["eval_every"])
patience = int(config["patience"])

best_hr = -1.0
best_epoch = 0
best_state = None
patience_counter = 0

start_time = time.perf_counter()

for epoch in range(epochs):
    model.train()

    total_loss = 0.0
    num_batches = max(1, len(user_item_pairs) // batch_size)

    perm = np.random.permutation(len(user_item_pairs))
    user_item_pairs_shuffled = user_item_pairs[perm]

    epoch_start = time.perf_counter()

    progress = tqdm(range(num_batches), desc=f"Epoch {epoch+1}/{epochs}", leave=False)
    for batch_idx in progress:
        start_idx = batch_idx * batch_size
        end_idx = start_idx + batch_size

        batch_pairs = user_item_pairs_shuffled[start_idx:end_idx]
        users_np = batch_pairs[:, 0]
        pos_items_np = batch_pairs[:, 1]

        neg_items_np = sample_neg_vectorized(users_np, n_items, user_pos, n_negs=n_negs)

        users = torch.LongTensor(users_np).to(device)
        pos_items = torch.LongTensor(pos_items_np).to(device)
        neg_items = torch.LongTensor(neg_items_np).to(device)

        # Forward with current parameters (one forward per batch)
        user_emb_propagated, item_emb_propagated = model(train_adj_torch)
        user_emb_ego, item_emb_ego = model.get_ego_embeddings()

        loss = bpr_loss_multi_neg(
            users,
            pos_items,
            neg_items,
            user_emb_propagated,
            item_emb_propagated,
            user_emb_ego,
            item_emb_ego,
            reg_weight=reg,
        )

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += float(loss.item())

        if (batch_idx + 1) % 10 == 0:
            progress.set_postfix(loss=f"{loss.item():.4f}", avg_loss=f"{total_loss/(batch_idx+1):.4f}")

    progress.close()

    avg_loss = total_loss / num_batches
    epoch_dur = time.perf_counter() - epoch_start

    do_eval = (len(val_items_dict) > 0) and ((epoch + 1) % eval_every == 0 or epoch == 0)
    if do_eval:
        model.eval()
        with torch.no_grad():
            user_emb_eval, item_emb_eval = model(train_adj_torch)
            user_emb_eval = user_emb_eval.cpu().numpy()
            item_emb_eval = item_emb_eval.cpu().numpy()

        val_metrics = evaluate_topk(user_emb_eval, item_emb_eval, val_items_dict, user_pos, top_k=20)
        val_hr = val_metrics['HR']

        elapsed = time.perf_counter() - start_time
        eta = (epochs - epoch - 1) * (elapsed / (epoch + 1))

        print(
            f"Epoch {epoch+1:3d}/{epochs} | Loss: {avg_loss:.4f} | "
            f"Val HR@20: {val_hr:.4f} | {epoch_dur:.1f}s | ETA: {eta/60:.1f}m"
        )

        if val_hr > best_hr:
            best_hr = val_hr
            best_epoch = epoch + 1
            best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print(f"\n⚠️  Early stopping at epoch {epoch+1} (patience={patience})")
            break

# Load best model
if best_state is not None:
    model.load_state_dict(best_state)
    print(f"\n✅ Best model from epoch {best_epoch} (Val HR@20: {best_hr:.4f})")
else:
    print("\n⚠️  No validation available; best_state not set. Model is last epoch.")

# ============================================================================
# FINAL EMBEDDINGS (NO NORMALIZATION)
# ============================================================================

print("\n" + "="*70)
print("EXTRACTING EMBEDDINGS")
print("="*70)

model.eval()
with torch.no_grad():
    user_emb_final, item_emb_final = model(train_adj_torch)

user_emb_final = user_emb_final.cpu().numpy()
item_emb_final = item_emb_final.cpu().numpy()

np.save('gcn_user_embeddings.npy', user_emb_final)
np.save('gcn_item_embeddings.npy', item_emb_final)

print(f"💾 Saved raw embeddings (NO normalization)")
print(f"   User: {user_emb_final.shape} | Item: {item_emb_final.shape}")
print(f"⏱️  Total training time: {(time.perf_counter() - start_time)/60:.2f} min")

# ============================================================================
# EVALUATION (TEST ONLY, ONCE) - RANKING METRICS
# ============================================================================

print("\n" + "="*70)
print("EVALUATION - TEST RANKING METRICS")
print("="*70)

# Build test ground truth
test_items_dict = build_items_dict(test_df)

# Evaluate at multiple K
for k in [10, 20, 50]:
    metrics = evaluate_topk(user_emb_final, item_emb_final, test_items_dict, user_pos, top_k=k)

    print(f"\n📊 Top-{k} Results:")
    print(f"   HR@{k}:        {metrics['HR']:.4f}")
    print(f"   Precision@{k}: {metrics['Precision']:.4f}")
    print(f"   Recall@{k}:    {metrics['Recall']:.4f}")
    print(f"   NDCG@{k}:      {metrics['NDCG']:.4f}")

print(f"\n✅ Evaluation complete on {len(test_items_dict)} test users")
print(f"✅ Users with ≤5 interactions excluded from evaluation: {users_with_5_or_less}")
print("\n" + "="*70)
print("🎯 LIGHTGCN COMPLETE")
print("="*70)
print("\n✓ 80:20 train-test split per user (>5 interactions)")
print("✓ Item-aware fix applied before validation split")
print("✓ Validation split from train only (no tuning on test)")
print("✓ Early stopping uses validation HR@20")
print("✓ Multi-negative BPR supported via n_negs")
print("✓ Ranking metrics: Precision@K, Recall@K, NDCG@K")
print("✓ No data leakage (train items masked during evaluation)")